In [71]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gmaps
from pprint import pprint
import requests
import json

In [72]:
from config import gmaps_key
gmaps.configure(api_key=gmaps_key)

In [73]:
tour_file = pd.read_csv('tour_schedule.csv')
tour_file

,Tour,City
0,Spring 2007,"Farmington, New Mexico, USA"
1,Spring 2007,"Tucson, Arizona, USA"
2,Spring 2007,"Dallas, Texas, USA"
3,Spring 2007,"Baton Rouge, Louisiana, USA"
4,Spring 2007,"New Orleans, Louisiana, USA"
...,...,...
457,Fall 2019,"Nyon, Switzerland"
458,Fall 2019,"Wartenberg, Germany"
459,Spring 2020,"Denver, Colorado, USA"
460,Spring 2020,"Flagstaff, Arizona, USA"


In [74]:
tour_file['State'] = ''
tour_file['Country'] = ''
tour_file['Cast'] = ''
for index, row in tour_file.iterrows():
    splitd = row['City'].split(', ')
    if len(splitd) == 1:
        tour_file.loc[index, 'State'] = '-'
        tour_file.loc[index, 'City'] = splitd[0]
        tour_file.loc[index, 'Country'] = splitd[0]
    elif len(splitd) == 2:
        tour_file.loc[index, 'State'] = '-'
        tour_file.loc[index, 'City'] = splitd[0]
        tour_file.loc[index, 'Country'] = splitd[1]
    elif len(splitd) == 3:
        tour_file.loc[index, 'State'] = splitd[1]
        tour_file.loc[index, 'City'] = splitd[0]
        tour_file.loc[index, 'Country'] = splitd[2]
    splitt = row['Tour'].split(' ')
    if splitt[0] == 'Spring':
        cast = 'A'
    elif splitt[0] == 'Fall':
        cast = 'B'
    tour_file.loc[index, 'Cast'] = cast + splitt[1][-2] + splitt[1][-1] 
tour_file.head()

,Tour,City,State,Country,Cast
0,Spring 2007,Farmington,New Mexico,USA,A07
1,Spring 2007,Tucson,Arizona,USA,A07
2,Spring 2007,Dallas,Texas,USA,A07
3,Spring 2007,Baton Rouge,Louisiana,USA,A07
4,Spring 2007,New Orleans,Louisiana,USA,A07


In [75]:
tour_file

,Tour,City,State,Country,Cast
0,Spring 2007,Farmington,New Mexico,USA,A07
1,Spring 2007,Tucson,Arizona,USA,A07
2,Spring 2007,Dallas,Texas,USA,A07
3,Spring 2007,Baton Rouge,Louisiana,USA,A07
4,Spring 2007,New Orleans,Louisiana,USA,A07
...,...,...,...,...,...
457,Fall 2019,Nyon,-,Switzerland,B19
458,Fall 2019,Wartenberg,-,Germany,B19
459,Spring 2020,Denver,Colorado,USA,A20
460,Spring 2020,Flagstaff,Arizona,USA,A20


In [76]:
tour_file['Country'].value_counts()

USA                    216
Mexico                  88
Germany                 22
Sweden                  20
Belgium                 19
Switzerland             16
Netherlands             12
Denmark                 11
Philippines              9
Italy                    9
Finland                  7
Taiwan                   7
México                   6
Thailand                 5
Poland                   2
Cuba                     2
Kosovo                   1
Norway                   1
Liechtenstein            1
Bermuda                  1
Genk Belgium             1
California USA           1
Estonia                  1
Canada                   1
Czech Republic           1
Grand Bahama Island      1
Lichtenstein             1
Name: Country, dtype: int64

In [77]:
cast = input('What cast did you travel in?')
cast_tour = tour_file.loc[tour_file['Cast'] == cast]

What cast did you travel in?A19


In [78]:
url = f'https://maps.googleapis.com/maps/api/geocode/json?key={gmaps_key}'
cast_tour['Lat'] = ''
cast_tour['Long'] = ''
for index, row in cast_tour.iterrows():
    if row['State'] != '-':
        append_url = f'&address=' + str(row['City']) + '%20' + str(row['State']) + '%20' + str(row['Country'])
    else:
        append_url = f'&address=' + str(row['City']) + '%20' + str(row['Country'])
    response = requests.get(url+append_url).json()
    cast_tour.loc[index, 'Lat'] = response['results'][0]['geometry']['location']['lat']
    cast_tour.loc[index, 'Long'] = response['results'][0]['geometry']['location']['lng']
    
        
cast_tour

C:\Users\dpatc\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\dpatc\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\dpatc\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

,Tour,City,State,Country,Cast,Lat,Long
430,Spring 2019,Greeley,CO,USA,A19,40.4233,-104.709
431,Spring 2019,Eerie,CO,USA,A19,40.0503,-105.05
432,Spring 2019,Grand Junction,CO,USA,A19,39.0639,-108.551
433,Spring 2019,Las Vegas,NV,USA,A19,36.1699,-115.14
434,Spring 2019,Sacramento/Modesto,CA,USA,A19,37.669,-120.913
435,Spring 2019,Olympia,WA,USA,A19,47.0379,-122.901
436,Spring 2019,Celaya,-,Mexico,A19,20.528,-100.811
437,Spring 2019,Naucalpan,-,Mexico,A19,19.4631,-99.2453
438,Spring 2019,Metapec,-,Mexico,A19,19.2529,-99.5986
439,Spring 2019,Mexico City,-,Mexico,A19,19.4326,-99.1332


In [79]:
marker_locations = cast_tour[['Lat', 'Long']]

info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>State</dt><dd>{State}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
tour_info = [info_box_template.format(**row) for index, row in cast_tour.iterrows()]
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations, info_box_content=tour_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))